# core

> Fill in a module description here

In [ ]:
#| default_exp snowflake

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from triggerkit import core
import snowflake.connector
from snowflake.connector.connection import SnowflakeConnection
from snowflake.connector.cursor import SnowflakeCursor
from typing import Dict, List, Callable, Optional, Any, Union

In [ ]:
#| export

core.VIEW_REGISTRY: Dict[str, Dict[str, Any]] = {}

def register_view(name: str, query: str, metadata: Optional[Dict[str, Any]] = None):
    """
    Register a Snowflake view with the system.
    
    Args:
        name: Name of the view
        query: SQL query or view name to fetch datay
        metadata: Additional metadata about the view
        
    Returns:
        None
    """
    core.VIEW_REGISTRY[name] = {
        'query': query,
        'metadata': metadata or {}
    }
    logger.info(f"Registered view: {name}")

In [ ]:
#| export 

core.snowflake_connection = None

def connect(config: Dict[str, Any]) -> SnowflakeConnection:
    """
    Establish connection to Snowflake.
    
    Args:
        config: Dictionary containing Snowflake connection parameters
        
    Returns:
        Snowflake connection object
    """
    # Check for environment variables for sensitive data
    password = os.environ.get('SNOWFLAKE_PASSWORD') or config.get('password')
    
    try:
        connection = snowflake.connector.connect(
            user=config.get('user'),
            password=password,
            account=config.get('account'),
            warehouse=config.get('warehouse'),
            database=config.get('database'),
            schema=config.get('schema'),
            role=config.get('role', 'ACCOUNTADMIN')
        )
        logger.info("Successfully connected to Snowflake")
        return connection
    except Exception as e:
        logger.error(f"Failed to connect to Snowflake: {str(e)}")
        raise

In [ ]:
#| export

def query(view_name: str) -> List[Dict[str, Any]]:
    """
    Execute a query for a registered view and return results.
    
    Args:
        view_name: Name of the registered view
        
    Returns:
        List of result rows as dictionaries
    """
    if view_name not in core.VIEW_REGISTRY:
        raise ValueError(f"View '{view_name}' is not registered")
    
    if not core.snowflake_connection:
        raise ValueError("Snowflake connection not established")
    
    view_config = core.VIEW_REGISTRY[view_name]
    query = view_config['query']
    
    try:
        cursor: SnowflakeCursor = core.snowflake_connection.cursor()
        cursor.execute(query)
        column_names = [col[0] for col in cursor.description]
        
        # Convert results to list of dictionaries
        results = []
        for row in cursor:
            results.append(dict(zip(column_names, row)))
        
        cursor.close()
        logger.info(f"Successfully executed query for view '{view_name}', returned {len(results)} rows")
        return results
    except Exception as e:
        logger.error(f"Failed to execute query for view '{view_name}': {str(e)}")
        raise

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()